In [ ]:
train_dir = "../input/brain-tumor-mri-dataset/Training"
test_dir = "../input/brain-tumor-mri-dataset/Testing"

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

train_results = model.train(
    data=train_dir,
    epochs=100,
    imgsz=640,
)

In [ ]:
import numpy as np
from mealpy import FloatVar, human_based, swarm_based
from ultralytics import YOLO

def objective_function(solution):   
    solution = convert_to_python_types(solution)
    
    learning_rate, batch_size, weight_decay, conf_thresh, iou_thresh, epochs = solution
    batch_size = int(batch_size)
    epochs = int(epochs)  

    model = YOLO("yolov8n.pt")

    try:
        results = model.train(
            data= train_dir,
            epochs=epochs,  
            imgsz=640,
            conf=conf_thresh,
            iou=iou_thresh,
            lr0=learning_rate,
            batch=batch_size,
            weight_decay=weight_decay,
        )

        val_results = model.val()

        map50 = val_results.metrics.box['map50']  
        map = val_results.metrics.box['map']      
        precision = val_results.metrics.box['precision']
        
        combined_score = (map50 + map + precision) / 3

        return -combined_score 
    
    except Exception as e:
        print(f"Optimization error: {e}")
        return float('inf')  

problem_dict = {
    "bounds": FloatVar(
        lb=(1e-5, 16, 1e-6, 0.1, 0.1, 5),   
        ub=(1e-1, 64, 1e-2, 0.9, 0.9, 50),  
        name=["learning_rate", "batch_size", "weight_decay", "conf_thresh", "iou_thresh", "epochs"]
    ),
    "minmax": "min",
    "obj_func": objective_function,
}


chio_model = human_based.CHIO.DevCHIO(
    epoch=20,             
    pop_size=30,          
    brr=0.2,              
    max_age=5         
)

chio_result = chio_model.solve(problem_dict)

best_solutions = [convert_to_python_types(chio_result.solution)]

lyrebird_model = swarm_based.LYO.BaseLYO(
    epoch=15,             
    pop_size=20,          
    c1=2.0,               
    c2=2.0,               
    w_min=0.5,            
    w_max=0.9             
)

lyrebird_result = lyrebird_model.solve(problem_dict, initial_solutions=best_solutions)

final_solution = convert_to_python_types(lyrebird_result.solution)

print("Final Solution (Python types):")
for name, value in zip(["learning_rate", "batch_size", "weight_decay", "conf_thresh", "iou_thresh", "epochs"], final_solution):
    print(f"{name}: {value}")
print(f"Final Fitness (Validation Loss): {lyrebird_result.target.fitness}")


model = YOLO("yolov8n-seg.pt")

model.train(
    data=train_dir,
    epochs=int(final_solution[5]),  
    imgsz=640,
    batch=int(final_solution[1]),
    lr0=final_solution[0],
    weight_decay=final_solution[2],
    conf=final_solution[3],
    iou=final_solution[4]
)

torch.save(model.state_dict(), 'models\brain_detect.pt')
